In [ ]:
import os
import pathlib
import pickle

import httpx
from dotenv import load_dotenv
from google import genai
from google.genai import types as gemini_types

load_dotenv(dotenv_path="../../.env")

aistudio_gemini_api_key = os.environ['GOOGLE_API_KEY']
print(aistudio_gemini_api_key[:3] + '...' + aistudio_gemini_api_key[-1:])
gemini_client = genai.Client(api_key=aistudio_gemini_api_key)

MODEL_NAME = "gemini-3-flash-preview"


# MODEL_NAME = "gemini-3-pro-preview"
# vertex_gemini_key = os.environ["VERTEX_AI_KEY"]
# headers = {"Authorization": f"Basic {vertex_gemini_key}"}
# base_url = "https://generative-ai-proxy.rcp.us-east-1.data.test.exp-aws.net/v1/proxy/vertex-ai"

# http_options=gemini_types.HttpOptions(
#         headers=headers,
#         api_version="",
#         base_url=base_url,
#         httpx_client=httpx.Client(verify=False)
#     )


# gemini_client = genai.Client(
#     api_key="dummy_value_to_by_pass_validation",
#     http_options=http_options
# )



In [15]:
from eduly import EdulyClient

eduly_client = EdulyClient(gemini_client=gemini_client)


In [ ]:
# # takes ~30 seconds
# breakdown_obj, raw_breakdown_response = eduly_client.breakdown(
#     file_path=pathlib.Path("./attention_is_all_you_need.pdf"),
#     model=MODEL_NAME,
#     thinking_level="high"
# )

# with open('./attention_is_all_you_need_gemini_3_flash_breakdown.pkl', 'wb') as f:
#     pickle.dump(raw_breakdown_response, f)

In [16]:
from eduly.models import Breakdown

with open('./attention_is_all_you_need_gemini_3_flash_breakdown.pkl', 'rb') as f:
    raw_breakdown_response = pickle.load(f)

breakdown_obj = Breakdown.model_validate_json(raw_breakdown_response.text)

In [17]:
topics = breakdown_obj.topics
for topic in topics:
    print('--------------------------------')
    print(topic.to_text())
    print('--------------------------------')



--------------------------------
**name**: Beyond Recurrence: The Motivation for Transformers

**summary**: Before Transformers, sequence modeling relied on Recurrent Neural Networks (RNNs) that processed data one step at a time, creating a massive bottleneck for parallelization. The Transformer was designed to break this sequential constraint, allowing models to process entire sequences simultaneously.

**full_explanation**:
In 2017, the dominant models for sequence tasks like translation were **Recurrent Neural Networks (RNNs)**, such as LSTMs and GRUs. These models process data sequentially: to calculate the state of the current word, the model must first calculate the state of the previous word ($h_{t-1}$). This inherent sequential nature prevents parallelization during training, which becomes a critical problem as datasets and sequence lengths grow. While factorization tricks and conditional computation provided some speedups, the fundamental constraint of sequential processing re

In [18]:
from tqdm import tqdm

i=0

for topic in tqdm(topics):
    storyboard_obj, raw_storyboard_response = eduly_client.storyboard(
        topic=topic,
        # source_file=pathlib.Path("./attention_is_all_you_need.pdf")
    )

    with open(f'./attention_is_all_you_need_gemini_3_flash_storyboard_{i}.pkl', 'wb') as f:
        pickle.dump(raw_storyboard_response, f)
    i+=1


  0%|          | 0/8 [00:00<?, ?it/s]


AttributeError: 'str' object has no attribute 'get'

In [12]:
from eduly.models import TopicStoryboard

with open('./attention_is_all_you_need_gemini_3_flash_storyboard_1.pkl', 'rb') as f:
    raw_storyboard_response = pickle.load(f)

storyboard_obj = TopicStoryboard.model_validate_json(raw_storyboard_response.text)

In [13]:
print(storyboard_obj.topic_name)
print(f"Visual Concept: {storyboard_obj.visual_concept}\n")
scenes = storyboard_obj.scenes
for scene in scenes:
    print(f"{scene.to_text()}")


The Core Mechanism: Scaled Dot-Product Attention
Visual Concept: The search-and-retrieval metaphor: A blue 'Query' vector acts as a spotlight, scanning across golden 'Key' vectors to determine which green 'Value' vectors should be retrieved and blended into the final output.

## [hook] The Library of Vectors
On a dark background, three distinct clusters of colorful boxes appear. The first cluster, labeled 'Queries' (Q), is a set of deep blue 1x4 horizontal rectangles. The second, labeled 'Keys' (K), is a set of golden yellow 1x4 horizontal rectangles. The third, 'Values' (V), consists of emerald green 1x4 horizontal rectangles. A single blue Query vector (q1) floats to the center. Below it, three golden Key vectors (k1, k2, k3) are lined up horizontally. The Query vector pulses, and faint gray arrows point from it to each Key, suggesting a search process. Text appears: 'What are we looking for? (Query) vs. What information is available? (Key)'.

## [mid] Measuring Similarity: The Dot P